In [1]:
import sys
sys.path.append('../src')

import typing as tp
from copy import copy

import numpy as np
import matplotlib.pyplot as plt
import torch
from omegaconf import OmegaConf

from deconv.classic.wiener.wiener import wiener_gray
from deconv.neural.usrnet.predictor import USRNetPredictor
from deconv.neural.dwdn.predictor import DWDNPredictor
from deconv.neural.kerunc.predictor import KerUncPredictor
from services.tester import MainTester


np.random.seed(8)

In [2]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

## 1 Скорость работы

In [3]:
NOISE_MU = 0
NOISE_STD = 0.01
IMAGE_SIZE = 256

In [17]:
images, psf = MainTester.get_data(
    image_path='../datasets/gt/precomp/city/01 (43).jpg',
    psf_path='../datasets/kernels/motion-blur/processed/synthetic/synthetic-18.npy',
    image_size=IMAGE_SIZE,
    noise_mu=NOISE_MU,
    noise_std=NOISE_STD,
)

_, psf_big = MainTester.get_data(
    image_path='../datasets/gt/precomp/city/01 (43).jpg',
    psf_path='../datasets/kernels/eye-psf/processed/synthetic/big-psf-1.npy',
    image_size=IMAGE_SIZE,
    noise_mu=NOISE_MU,
    noise_std=NOISE_STD,
)

In [18]:
blurred_1d = images['linrgb_float']['blurred_no_noise']['1d']
blurred_3d = images['linrgb_float']['blurred_no_noise']['3d']
blurred_1d.shape, psf.shape, psf_big.shape

((256, 256), (41, 41), (256, 256))

### **CPU:**

In [20]:
config_gpu = OmegaConf.load('../configs/config_viz.yml')
cm = config_gpu.models
cd = config_gpu.dataset

models = dict()

models['wiener'] = dict()
models['wiener']['noise'] = lambda image, psf: wiener_gray(image, psf, **cm.wiener_nonblind_noise.noise_params)
models['wiener']['no_noise'] = lambda image, psf: wiener_gray(image, psf, **cm.wiener_nonblind_noise.no_noise_params)

models['kerunc'] = dict()
models['kerunc']['noise'] = KerUncPredictor(**cm.kerunc.noise_params)
models['kerunc']['no_noise'] = KerUncPredictor(**cm.kerunc.no_noise_params)

models['usrnet'] = dict()
models['usrnet']['noise'] = USRNetPredictor(model_path=cm.usrnet.model_path, **cm.usrnet.noise_params)
models['usrnet']['no_noise'] = USRNetPredictor(model_path=cm.usrnet.model_path, **cm.usrnet.no_noise_params)

models['dwdn'] = dict()
models['dwdn']['noise'] = DWDNPredictor(model_path=cm.dwdn.model_path, **cm.dwdn.noise_params)
models['dwdn']['no_noise'] = DWDNPredictor(model_path=cm.dwdn.model_path, **cm.dwdn.no_noise_params)


In [21]:
wiener = models['wiener']['noise']
dwdn = models['dwdn']['noise']
kerunc = models['kerunc']['noise']
usrnet = models['usrnet']['noise']

#### PSF 41x41

In [8]:
%timeit -n 10 -r 3 wiener(blurred_1d, psf)

2.71 ms ± 65.8 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)


In [9]:
%timeit -n 10 -r 3 kerunc(blurred_1d, psf)

1.87 s ± 8.84 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)


In [10]:
%timeit -n 10 -r 3 dwdn(blurred_3d, psf)

3.14 s ± 18.8 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)


In [11]:
%timeit -n 10 -r 3 usrnet(blurred_3d, psf)

2.2 s ± 5.86 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)


#### PSF 256x256

In [22]:
%timeit -n 1 -r 1 wiener(blurred_1d, psf_big)

58.2 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [23]:
%timeit -n 1 -r 1 kerunc(blurred_1d, psf_big)

1.77 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [26]:
%timeit -n 1 -r 1 dwdn(blurred_3d, psf_big)

14.6 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [25]:
%timeit -n 1 -r 1 usrnet(blurred_3d, psf_big)

2.23 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### **GPU:**

In [27]:
config_gpu = OmegaConf.load('../configs/config_viz_cuda.yml')
cm = config_gpu.models
cd = config_gpu.dataset

models_gpu = dict()

models_gpu['kerunc'] = dict()
models_gpu['kerunc']['noise'] = KerUncPredictor(**cm.kerunc.noise_params)
models_gpu['kerunc']['no_noise'] = KerUncPredictor(**cm.kerunc.no_noise_params)

models_gpu['usrnet'] = dict()
models_gpu['usrnet']['noise'] = USRNetPredictor(model_path=cm.usrnet.model_path, **cm.usrnet.noise_params)
models_gpu['usrnet']['no_noise'] = USRNetPredictor(model_path=cm.usrnet.model_path, **cm.usrnet.no_noise_params)

models_gpu['dwdn'] = dict()
models_gpu['dwdn']['noise'] = DWDNPredictor(model_path=cm.dwdn.model_path, **cm.dwdn.noise_params)
models_gpu['dwdn']['no_noise'] = DWDNPredictor(model_path=cm.dwdn.model_path, **cm.dwdn.no_noise_params)


In [28]:
dwdn   = models_gpu['dwdn']['noise']
kerunc = models_gpu['kerunc']['noise']
usrnet = models_gpu['usrnet']['noise']

#### PSF 41x41

In [14]:
%timeit -n 10 -r 3 kerunc(blurred_1d, psf)

108 ms ± 21.1 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)


In [15]:
%timeit -n 10 -r 3 dwdn(blurred_3d, psf)

51.4 ms ± 429 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)


In [16]:
%timeit -n 10 -r 3 usrnet(blurred_3d, psf)

77.2 ms ± 145 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)


#### PSF 256x256

In [29]:
%timeit -n 10 -r 3 kerunc(blurred_1d, psf_big)

97 ms ± 6.37 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)


In [30]:
%timeit -n 10 -r 3 dwdn(blurred_3d, psf_big)

124 ms ± 556 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)


In [31]:
%timeit -n 10 -r 3 usrnet(blurred_3d, psf_big)

77.1 ms ± 163 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)
